<a href="https://colab.research.google.com/github/DanMatsu0/machine-learning-projectsV1/blob/main/mnist_cnn_tensorflow_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import

In [1]:
import tensorflow as tf
#V3追加
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

データセットの読み込み

In [2]:
# データセットのロード
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [3]:
# データの前処理
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

#V2追加
# データセットをバッチ化
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(64)



In [4]:
# 出力結果の固定
tf.random.set_seed(0)


#v3
# K-Fold 交差検証
k = 5  # K-Fold 交差検証の K を設定
scores = []  # 各 Fold の精度を格納するリスト

# 各 Fold でモデルを訓練・評価
for train_index, val_index in KFold(n_splits=k, shuffle=True).split(x_train, y_train):
    x_train_fold, x_val_fold = x_train[train_index], x_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    # データセットをバッチ化
    train_fold_dataset = tf.data.Dataset.from_tensor_slices((x_train_fold, y_train_fold)).batch(64)
    val_fold_dataset = tf.data.Dataset.from_tensor_slices((x_val_fold, y_val_fold)).batch(64)

    # モデルの定義
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(6, (5, 5), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(16, (5, 5), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
    ])

    # モデルのコンパイル
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # モデルの訓練
    model.fit(train_fold_dataset, epochs=5, steps_per_epoch=len(train_fold_dataset))

    # モデルの評価
    val_loss, val_acc = model.evaluate(val_fold_dataset)
    test_loss, test_acc = model.evaluate(test_dataset)
    print('Val accuracy:', val_acc)
    print('Test accuracy:', test_acc)
    scores.append(val_acc)

# 平均精度を計算
print('平均精度:', sum(scores) / len(scores))



Epoch 1/5
750/750 [==============================] - 6s 3ms/step - loss: 0.3053 - accuracy: 0.9112
Epoch 2/5
750/750 [==============================] - 2s 3ms/step - loss: 0.0937 - accuracy: 0.9714
Epoch 3/5
750/750 [==============================] - 2s 3ms/step - loss: 0.0665 - accuracy: 0.9793
Epoch 4/5
750/750 [==============================] - 2s 3ms/step - loss: 0.0519 - accuracy: 0.9842
Epoch 5/5
157/157 [==============================] - 0s 2ms/step - loss: 0.0488 - accuracy: 0.9836
Val accuracy: 0.9822499752044678
Test accuracy: 0.9836000204086304
Epoch 1/5
750/750 [==============================] - 3s 3ms/step - loss: 0.2769 - accuracy: 0.9196
Epoch 2/5
750/750 [==============================] - 2s 3ms/step - loss: 0.0906 - accuracy: 0.9726
Epoch 3/5
750/750 [==============================] - 2s 3ms/step - loss: 0.0642 - accuracy: 0.9805
Epoch 4/5
750/750 [==============================] - 2s 3ms/step - loss: 0.0507 - accuracy: 0.9837
Epoch 5/5
157/157 [=======================